In [ ]:
pip install git+https://github.com/Commonists/pageview-api.git

In [ ]:
pip install wikipedia

In [ ]:
# Run only for the first time #
import csv
f = open('WikiDataSet.csv', 'w', newline = '', encoding="utf-8") #open the csv at the first time 
writer = csv.writer(f)
col_list = ['Page Title', 'URL', 'Num Of Pictures', 'Num Of Languages', 'Num Of References', 'Num Of Contents','Num Of Tables', 'Num Of Categories', 'Num Of Redirects', 'Views In 30 Last Days', 'Avg Views Per Day', 'Total Views', 'Num Of Edits']
writer.writerow(col_list)
f.close()

In [ ]:
import csv
url = "http://en.wikipedia.org/wiki/Special:Random" #Random page picker from Wikipedia
workers_num = 16 #Parallel processors
requests_per_time = 1 #Requests per second
time = 1 #second
url_list = []

In [ ]:
#Imports
import threading #Run all processors simultaneously
from queue import Queue #Processor Queue
from threading import Semaphore, Timer #Time and semaphor to approve or restrict CPU access
import html #html request
import sys
import csv
from bs4 import BeautifulSoup
import requests
import wikipedia
import pageviewapi
import pageviewapi.period


#set parameters
workers_num = int(workers_num) #Number of processors at the same time
requests_per_time = int(requests_per_time) #Requests per second
time = int(time)#second

link = url #Random address

#semaphore for sync printing
print_semaphore = Semaphore(1)

#producer(scheduler) architecture
class Producer:
    print("IN Producer")
    def __init__(self, queue):
        self.queue = queue
    def run(self):
        threading.Timer(time, self.run).start()
        # print("Reset")
        for i in range(requests_per_time):
            self.queue.put(i)
            
#consumers(workers) architecture
class Consumer:
    print("IN CONSUMER")
    def __init__(self, queue, id):
        self.queue = queue
        self.__id = id
    def run(self):
        #while True:
        for i in range(100):
            try:
                    #check if has permission to send a request (based on requests_per_time) if yes continue, else blocked until producer initiates new permissions
                self.queue.get()
                    #scrape and search keywords
                    #sync printing
                print_semaphore.acquire()
                url = Scraper.wiki_scrape(link)
                print("Worker: "+str(self.__id)) #Printing which processor it runs on
                print("Random URL: "+ str(url)) #Printing the URL
                print("-----------------------------")
                print_semaphore.release()
                self.queue.task_done()
            except KeyboardInterrupt:
                print('Interrupted')
                sys.exit()

#scrape a random wiki page
class Scraper:
    def wiki_scrape(url): 
        print("in wiki")
        url_list = []
        res = requests.get(url,)
        res.raise_for_status() #Checking for an error during the process
        wiki = BeautifulSoup(res.text,"html.parser")
        print("URL: ", res.url)           
        
        title = wiki.find("h1" , attrs = {"class":"firstHeading"})#Page Title
        url_list.append(title.text)
        
        url_list.append(str(res.url))#URL
        
        pic_counter = wiki.find_all("a" , attrs = {"class":"image"})#Num Of Pictures
        url_list.append(len(pic_counter))
        
        languages_counter = wiki.find_all("a" , attrs = {"class":"interlanguage-link-target"})#Num Of Languages
        url_list.append(len(languages_counter))
        
        references_counter = wiki.find_all("span" , attrs = {"class":"reference-text"})#Num Of References
        url_list.append(len(references_counter))
        
        contents_counter = wiki.find_all("li" , attrs = {"class":"toclevel-1"})#Num Of Contents
        url_list.append(len(contents_counter))
        
        tables_counter = wiki.find_all("table" , attrs = {"class":"wikitable"})#Num Of Tables
        url_list.append(len(tables_counter))
        
        categories_counter = wiki.find("div" , attrs = {"class":"mw-normal-catlinks"}).find_all('li') #Num Of Categories
        url_list.append(len(categories_counter))

        info_url = (wiki("ul" , attrs = {"class":"vector-menu-content-list"})[7].find_all('li'))[5].find('a').get('href')#page title
        html = 'https://en.wikipedia.org' + info_url #make the link
        html = requests.get(html)
        info_page = BeautifulSoup(html.text, "html.parser")#html
        
        Num_of_redirects_to_the_page = ((info_page("table" , attrs = {"class":"wikitable mw-page-info"})[0].find_all('tr'))[8].find_all('td'))[1]#Num Of Redirects
        url_list.append(Num_of_redirects_to_the_page.text)
        
        views_in_last_30 = info_page.find("div" , attrs = {"class":"mw-pvi-month"})#Views In 30 Last Days
        url_list.append(views_in_last_30.text)
        
        ##with API library (the last if from 7.1.2015 mm.dd.yy)##
        Avg_views_per_day = pageviewapi.period.sum_last('en.wikipedia', title.text, last=2373, access='all-access', agent='all-agents') / 2373 # Avg Views Per Day
        url_list.append(Avg_views_per_day)
        
        total_views = pageviewapi.period.sum_last('en.wikipedia', title.text, last=2373, access='all-access', agent='all-agents') #Total Views
        url_list.append(total_views)
        
        edits = ((info_page("table" , attrs = {"class":"wikitable mw-page-info"})[2].find_all('tr'))[4].find_all('td'))[1]#Num Of Edits
        url_list.append(edits.text)
        
        #Adds variables that will enter the list and print on the same line
        Scraper.csv_maker(url_list)
        
        return res.url
    
    def csv_maker(urls):
        print("in csv")
        f = open('WikiDataSet.csv', 'a', newline = '', encoding="utf-8") #Create the file
        writer = csv.writer(f)       
        writer.writerow(urls) 
        print("PRINTED")

        f.close() 

    
#create, start and join consumers(workers) and producer(scheduler) threads
def main():
    q = Queue(maxsize=requests_per_time) #Scheduled Queue
        #create and start producer thread
    scheduler = Producer(q)
    scheduler_thread = threading.Thread(target=scheduler.run)
    scheduler_thread.start() #Create and start scheduled threading
        #create and start consumers thread
    workers = [] 
    for worker_id in range(workers_num): #For each processor
        worker = Consumer(q, worker_id)
        workers.append(worker) #Add processor to processor list
    worker_threads = []
    for worker in workers:
        t = threading.Thread(target=worker.run)
        t.start()
        worker_threads.append(t)#Connect all threads and finish
        #join all threads and terminate
    scheduler_thread.join()
    for t in worker_threads:
        t.join()      
    print("DONE!")
      
main()

